In [19]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path
import yfinance as yf 
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from scipy.stats.mstats import winsorize
import os
import glob
import dateutil.parser as dparser


import scipy

In [20]:
"""Ledoit & Wolf constant correlation unequal variance shrinkage estimator."""
from typing import Tuple
import numpy as np

def shrinkage(returns: np.array) -> Tuple[np.array, float, float]:
    """Shrinks sample covariance matrix towards constant correlation unequal variance matrix.
    Ledoit & Wolf ("Honey, I shrunk the sample covariance matrix", Portfolio Management, 30(2004),
    110-119) optimal asymptotic shrinkage between 0 (sample covariance matrix) and 1 (constant
    sample average correlation unequal sample variance matrix).
    Paper:
    http://www.ledoit.net/honey.pdf
    Matlab code:
    https://www.econ.uzh.ch/dam/jcr:ffffffff-935a-b0d6-ffff-ffffde5e2d4e/covCor.m.zip
    Special thanks to Evgeny Pogrebnyak https://github.com/epogrebnyak
    :param returns:
        t, n - returns of t observations of n shares.
    :return:
        Covariance matrix, sample average correlation, shrinkage.
    """
    t, n = returns.shape
    mean_returns = np.mean(returns, axis=0, keepdims=True)
    returns -= mean_returns
    sample_cov = returns.transpose() @ returns / t

    # sample average correlation
    var = np.diag(sample_cov).reshape(-1, 1)
    sqrt_var = var ** 0.5
    unit_cor_var = sqrt_var * sqrt_var.transpose()
    average_cor = ((sample_cov / unit_cor_var).sum() - n) / n / (n - 1)
    prior = average_cor * unit_cor_var
    np.fill_diagonal(prior, var)

    # pi-hat
    y = returns ** 2
    phi_mat = (y.transpose() @ y) / t - sample_cov ** 2
    phi = phi_mat.sum()

    # rho-hat
    theta_mat = ((returns ** 3).transpose() @ returns) / t - var * sample_cov
    np.fill_diagonal(theta_mat, 0)
    rho = (
        np.diag(phi_mat).sum()
        + average_cor * (1 / sqrt_var @ sqrt_var.transpose() * theta_mat).sum()
    )

    # gamma-hat
    gamma = np.linalg.norm(sample_cov - prior, "fro") ** 2

    # shrinkage constant
    kappa = (phi - rho) / gamma
    shrink = max(0, min(1, kappa / t))

    # estimator
    sigma = shrink * prior + (1 - shrink) * sample_cov

    return sigma, average_cor, shrink

In [21]:
def shrinkage_EMW(returns_tmp: np.array, lookback = 126) -> Tuple[np.array, float, float]:
    """Shrinks sample covariance matrix towards constant correlation unequal variance matrix.
    Ledoit & Wolf ("Honey, I shrunk the sample covariance matrix", Portfolio Management, 30(2004),
    110-119) optimal asymptotic shrinkage between 0 (sample covariance matrix) and 1 (constant
    sample average correlation unequal sample variance matrix).
    Paper:
    http://www.ledoit.net/honey.pdf
    Matlab code:
    https://www.econ.uzh.ch/dam/jcr:ffffffff-935a-b0d6-ffff-ffffde5e2d4e/covCor.m.zip
    Special thanks to Evgeny Pogrebnyak https://github.com/epogrebnyak
    :param returns:
        t, n - returns of t observations of n shares.
    :return:
        Covariance matrix, sample average correlation, shrinkage.
    """
    returns = returns_tmp.tail(lookback).values
    t, n = returns.shape
    mean_returns = np.mean(returns, axis=0, keepdims=True) # make EWMA
    returns -= mean_returns
    COV_tmp = returns_tmp.ewm(span = lookback).cov()
    idx = returns_tmp.index.get_level_values(0)[-1]
    sample_cov = COV_tmp[COV_tmp.index.get_level_values(0) == idx]
    sample_cov = sample_cov.values
    #sample_cov = returns.transpose() @ returns / t

    # sample average correlation
    var = np.diag(sample_cov).reshape(-1, 1)
    sqrt_var = var ** 0.5
    unit_cor_var = sqrt_var * sqrt_var.transpose()
    average_cor = ((sample_cov / unit_cor_var).sum() - n) / n / (n - 1)
    prior = average_cor * unit_cor_var
    np.fill_diagonal(prior, var)

    # pi-hat
    y = returns ** 2
    phi_mat = (y.transpose() @ y) / t - sample_cov ** 2
    phi = phi_mat.sum()

    # rho-hat
    theta_mat = ((returns ** 3).transpose() @ returns) / t - var * sample_cov
    np.fill_diagonal(theta_mat, 0)
    rho = (
        np.diag(phi_mat).sum()
        + average_cor * (1 / sqrt_var @ sqrt_var.transpose() * theta_mat).sum()
    )

    # gamma-hat
    gamma = np.linalg.norm(sample_cov - prior, "fro") ** 2

    # shrinkage constant
    kappa = (phi - rho) / gamma
    shrink = max(0, min(1, kappa / t))

    # estimator
    sigma = shrink * prior + (1 - shrink) * sample_cov

    return sigma, average_cor, shrink

In [24]:
from scipy.stats import norm
import ezodf
import scipy.optimize as sco
import scipy

from sklearn.covariance import LedoitWolf

def Optimize_Portfolio(data ,lookback = 126, risk_free = 0, objective = 'Kelly'):

    ret = (data-1).mean()
    #cov_fit = LedoitWolf().fit(data)
    #cov = cov_fit.covariance_
    cov, average_cor, shrink = shrinkage_EMW(data, lookback = lookback)
    #cov = PCA_cov(data, N=5)
   
  
    if objective == 'Max Div':
        num_assets = len(data.columns)
        args = (cov)
        constraints = ({'type':'ineq', 'fun': lambda x: x},#all elements greater than one
                  #{'type':'ineq', 'fun': lambda x: 1 - np.sum(x)} # sum <= 1
                  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
        
        result = sco.minimize(calc_diversification_ratio, num_assets*[1./num_assets,], args=args, 
                              method='SLSQP', constraints=constraints, tol = 0.0000000000000000000000001)
        
    elif objective == "min var":
        num_assets = len(data.columns)
        args = (cov)
        constraints = ({'type':'ineq', 'fun': lambda x: x},#all elements greater than one
                  #{'type':'ineq', 'fun': lambda x: 1 - np.sum(x)} # sum <= 1
                  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
        
        result = sco.minimize(port_var, num_assets*[1./num_assets,], args=args, 
                              method='SLSQP', constraints=constraints, tol = 0.0000000000000000000000001)
    elif objective == "erc":
        num_assets = len(data.columns) 
        args = (cov)
        constraints = ({'type':'ineq', 'fun': lambda x: x},#all elements greater than one
                  #{'type':'ineq', 'fun': lambda x: 1 - np.sum(x)} # sum <= 1
                  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                      {'type':'ineq', 'fun': lambda x: x-(1/num_assets)*0.7}, # min position
                      {'type':'ineq', 'fun': lambda x: (1/num_assets)*1.3-x}) # max position
        
        result = sco.minimize(erc, num_assets*[1./num_assets,], args=args, 
                              method='SLSQP', constraints=constraints,
                                   options={'disp': True, 'maxiter': 100000, 'ftol' : 1e-100000,
                      'eps':1e-6}, tol = 0.0000000000000000000000001)
        

    return (result)




def port_var(weights, cov):
    var = weights.dot(cov).dot(weights)
    return(var)

def port_ret(weights, ret, risk_free = 0):
    #needs to be array
    ret = ret - risk_free
    port_ret = weights.dot(ret)
    return(port_ret)

def risk_parity(data):
    vol = np.log((data)).std()

    sum_vol = 0
    for i in range(len(vol)):
        sum_vol =sum_vol + (1/vol[i])
    
    weight = []
    for i in range(len(vol)):
        w = (1/vol[i])/(sum_vol)
        weight.append(w)
   
    weight = [round(num, 2) for num in weight]
    return(weight)




def calc_diversification_ratio(weights, cov):
    # average weighted vol
    w_vol = np.dot(np.sqrt(np.diag(cov)), weights.T)
    # portfolio vol
    port_vol = np.sqrt(port_var(weights, cov))
    
    diversification_ratio = w_vol/port_vol
    # return negative for minimization problem (maximize = minimize -)
    return -diversification_ratio

def erc(weights, cov):
        # these are non normalized risk contributions, i.e. not regularized
        # by total risk, seems to help numerically
        risk_contributions = np.dot(weights, cov) * weights
        a = np.reshape(risk_contributions, (len(risk_contributions), 1))
        # broadcasts so you get pairwise differences in risk contributions
        risk_diffs = a - a.transpose()
        sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
        # https://stackoverflow.com/a/36685019/1451311
        return sum_risk_diffs_squared #/ scale_factorcov
    


import sklearn.datasets, sklearn.decomposition

def PCA_cov(data, N = 5):
    
    X = data.ewm(span = 252).cov()
    DATE_IDX = X.index.get_level_values(level=0)[-1]
    X = X[X.index.get_level_values(0)==DATE_IDX].droplevel(0)
    mu = np.mean(X, axis=0)

    pca = sklearn.decomposition.PCA()
    pca.fit(X)

    nComp = N
    Xhat = np.dot(pca.transform(X)[:,:nComp], pca.components_[:nComp,:])
    Xhat += mu
    clean_cov = pd.DataFrame(Xhat)
    clean_cov.index = X.index
    clean_cov.columns = X.index
    return(clean_cov)

In [25]:
def ERC_gestalt(data, lookback = 126):
    
    prices_df = pd.DataFrame()
    for tick in data['Yahoo']:
    
        price = yf.download(tick,start='2000-01-01', progress = False, threads = False)
        price = price['Adj Close']
        prices_df[tick] = price
    
    log_ret = np.log(prices_df) - np.log(prices_df.shift(1))
    log_ret = log_ret.dropna()
    
    weight = Optimize_Portfolio(log_ret, lookback = lookback, objective='erc')['x'].round(3)

    return(weight)

def round_to_multiple(number, multiple):
    return multiple * round(number / multiple)

## Import old portfolios to construct staggerd portfolio

- Q: How to handel "hold" positions?
- "Hold" companies shold have "Min Position" == ACTION/3 rest is weighted from this? and MAX = Average posiotn

In [26]:
### Settings
START_DATE = '2016-01-01'
error_count = 0
error_list = []


latest_file= max(glob.glob("../equity_data/*.*"), key=os.path.getmtime)

signal_df = pd.read_excel(latest_file, sheet_name="Export")
signal_df = signal_df.rename({'Performance - Perform. 3m' : 'Return 3m','Performance - Perform. 6m' : 'Return 6m',
                            'Total Return - Return 1y' : 'Return 1y',
                            'Div. Yield - Current': 'Yield',
                            'Total Equity  - Millions':'Total Equity', 'FCF - Millions': 'FCF','ROE - Current':'ROE',
                            'Volatility - St.Dev. 100d':'Volatility','Market Cap - Current': 'Market Cap', 
                            'ROC - Current':'ROC', 'Tot. Assets - Millions':'Tot. Assets', 
                            'Gross profit - Millions':'Gross profit', 'Assets Turn - Current': 'Assets Turn',
                            'P/FCF - Current':'P/FCF', 'P/E - Current':'P/E', 'P/S - Current':'P/S',
                            'P/B - Current':'P/B','EV/EBIT - Current':'EV/EBIT',
                            'Info - Country' : 'Country','F-Score - Point':'F-Score',
                            'Info - List' : 'List', 'Info - Sector' : 'Sector', 'Info - Industry' : 'Industry',
                            'Info - Ticker' : 'Ticker', 'Info - Yahoo':'Yahoo', 'Info - Last Report': 'Last Report',
                           'Volume - Average 50d Mill' : 'Volume', 'Tot. Assets - Growth 1y' : 'Asset Growth'}, axis=1)


signal_df = signal_df.loc[ (signal_df['List'] != 'Spotlight') 
                        & (signal_df['List'] != 'NGM') & (signal_df['Country'] == "Sweden") &
                         (signal_df['Market Cap'] > 200)]

signal_df = signal_df.loc[(signal_df['Sector'] != 'Financials')]

# Set to dattime
signal_df['Last Report'] = pd.to_datetime(signal_df['Last Report'])
#set new index
signal_df.index = range(len(signal_df.index))


signal_df['Res_Mom_1M'] = np.nan
signal_df['Res_Mom_1M_alt'] = np.nan

signal_df['Tot_Mom_1M'] = np.nan
signal_df['Sea_month_5yr'] = np.nan
signal_df['idio_vol_20day'] = np.nan
signal_df['maxret_5days'] = np.nan
signal_df["EAR_std"]= np.nan
signal_df["5yr_vol"]= np.nan
signal_df["liq_shock"]= np.nan

index = yf.download('^OMXSPI',start=START_DATE, threads = False, progress = False)
index = index['Adj Close']
for i in tqdm(range(len(signal_df))):

    try:
        stock_tmp = yf.download(signal_df.iloc[i]['Yahoo'],start=START_DATE, progress = False, threads = False)

        
        stock = stock_tmp['Adj Close']
        import_data = pd.concat([stock, index], axis = 1)
        import_data.columns = ['stock', 'index']
        import_data = import_data.dropna()
        
        long_df = import_data.copy()
        ret_df = np.log(import_data/import_data.shift()).dropna()
        ### SEASONALITY
        
        monthly_df = import_data.resample('M').last()
        monthly_ret_df = np.log(monthly_df/monthly_df.shift()).dropna()
         
        ### 1 Month Momentum
        signal_df.loc[i,"Tot_Mom_1M"] = ret_df['stock'].tail(21).sum()
        
        
        ##EAR
        idx = ret_df.index.get_loc(signal_df.iloc[i]['Last Report'], method='nearest')
        
        EA_data = import_data.iloc[idx - 2 : idx +2 ]
        
        EA_ret = (EA_data.pct_change().dropna()+1).cumprod().tail(1)
        pead_ret = float(EA_ret['stock'] - EA_ret['index']) #Should use np.log()
        pead_vol = np.log(stock.iloc[:idx]/stock.iloc[:idx].shift()).tail(60).std()*252**.5
        signal_df.loc[i, 'EAR_std'] = pead_ret/pead_vol
        
        ## liquidity shock
        
        stock_volume = stock_tmp.copy()
        stock_volume['volume_sek'] = stock_volume['Close'] *stock_volume['Volume']
        
        # Resample to monthly for sobustness???
        stock_volume = stock_volume.rolling(21).sum().resample('30D').last()
        

        liq_shock = (stock_volume['volume_sek'].tail(1) - stock_volume['volume_sek'].tail(12).mean())/stock_volume['volume_sek'].tail(12).std()
        signal_df.loc[i,"liq_shock"] = float(liq_shock)
        
        ### RESIDUAL MOMENTUM
        ret_trim_df = ret_df.drop(ret_df.iloc[[idx - 1, idx, idx +1, idx +2 ]].index)
        
        
        signal_df.loc[i,"maxret_5days"] = np.mean(sorted(ret_trim_df['stock'].tail(21))[-5:])
        y_res = ret_trim_df.tail(21)['stock']
        X_res = np.array(ret_trim_df.tail(21)['index']).reshape(-1, 1)
        reg_res = LinearRegression().fit(X_res, y_res)
        residuals_res = y_res - reg_res.predict(X_res)
        signal_df.loc[i,"idio_vol_20day"] = residuals_res.std()
        
        #volume weight for short term rev
        vol_weight_tmp = stock_tmp.copy()
        vol_weight_tmp['volume_sek'] = vol_weight_tmp['Close'] *vol_weight_tmp['Volume']
        #weight by 60 day MA
        vol_weight = vol_weight_tmp['volume_sek'].rolling(252).mean() / vol_weight_tmp['volume_sek']
        
        reg_df = ret_trim_df.tail(3*252)
        ## Identify Report 
        if len(reg_df)>(2*252):
            y = reg_df['stock']
            X = np.array(reg_df['index']).reshape(-1, 1)
            reg = LinearRegression().fit(X, y)
            beta = reg.coef_[0]
            residuals = y - reg.predict(X)
            std_residuals = residuals/residuals.std()
            
            signal_df.loc[i,"Res_Mom_1M"] = std_residuals.tail(21).sum()# + np.log(vol_weight.tail(21)).sum()
           
        
        if len(monthly_ret_df)>=36:
            seas_list = []
            monthly_vol = 0
            for look in [12,24,36,48,60]:
                try:
                    seas_list.append(monthly_ret_df['stock'].iloc[-look])
                    monthly_vol = (monthly_ret_df['stock'].tail(60)+1).std() * np.sqrt(12)
                except:
                    pass
        
            signal_df.loc[i,"Sea_month_5yr"] = np.mean(seas_list) *12
            signal_df.loc[i,"5yr_vol"] = monthly_vol
            
            
    except:
        error_count = error_count + 1
        error_list.append(i)

 69%|██████▉   | 332/482 [01:56<00:42,  3.54it/s]


1 Failed download:
- SDIP.ST: No data found, symbol may be delisted


100%|██████████| 482/482 [02:50<00:00,  2.83it/s]


In [27]:
latest_file

'../equity_data/Borsdata_2022-08-27.xlsx'

In [28]:
#### CREATE NEW SECTORS!
signal_df.loc[signal_df['Industry'].isin(
    ['Leisure', 'Gambling & Casinos','Airlines','Hotels']),'Sector'] = 'Travel & Leisure'


signal_df.loc[signal_df['Industry'].isin(
    ['Pharmaceuticals']),'Sector'] = 'Pharmaceuticals'

signal_df.loc[signal_df['Industry'].isin(
    ['Medical Equipment']),'Sector'] = 'Medical Equipment'


signal_df.loc[signal_df['Industry'].isin(
    ['Retailers','Auto & Equipment','Industrial Components', 'Clothing & Footwear',
    'Consumer Electronics', 'Accessories' ]),'Sector'] = 'Retail'


signal_df.loc[signal_df['Industry'].isin(
    ['IT Consulting', 'IT Services', 'Communications',]),'Sector'] = 'Software'



signal_df.loc[signal_df['Industry'].isin(
    ['Industrial Components',
    'Energy & Recycling' ]),'Sector'] = 'General Industrials'



signal_df.loc[signal_df['Industry'].isin(
    ['Construction Supplies','Construction & Infrastructure',
     'Installation']),'Sector'] = 'Construction & Materials' 



signal_df.loc[signal_df['Industry'].isin(
    ['Industrial Machinery', 'Electrical Components']),'Sector'] = 'Electronic & Electrical Equipment'

signal_df.loc[signal_df['Industry'].isin(
    ['Holding Companies']),'Sector'] = 'Holding Companies'


signal_df.loc[signal_df['Industry'].isin(
    ['Real Estate']),'Sector'] = 'Real Estate'


In [29]:
#signal_df['Industry'].unique()

#signal_df

In [30]:
CUTOFF = 0.33 #0.25 # which cut off??

method = 'median'

### SHORT TERM REVERSAL - adjust for industry ###
signal_df['Res_Mom_1M_adj'] = signal_df["Res_Mom_1M"] - signal_df.groupby("Sector")["Res_Mom_1M"].transform(method)


#Vol adjsuted Seasonality
signal_df['Sea_month_5yr_std'] = signal_df['Sea_month_5yr']/signal_df['5yr_vol']

### IVOL - adjust for industry

signal_df['idio_vol_20day_adj'] = signal_df["idio_vol_20day"] - signal_df.groupby("Sector")["idio_vol_20day"].transform(method)


## MAX RET - adjust for industry
signal_df['maxret_5days_adj'] = signal_df["maxret_5days"] - signal_df.groupby("Sector")["maxret_5days"].transform(method)



########## INDUSTRY MOMENTUM ASSNESS SHOWS THAT EQUAL WEIGHT WORKS
signal_df['Sector Weighted Mom'] = signal_df.groupby("Sector")["Tot_Mom_1M"].transform(method)

# IMPUTE MEDIAN VALUE FOR NANS
signal_df['Res_Mom_1M_adj'] = signal_df['Res_Mom_1M_adj'].fillna(signal_df['Res_Mom_1M'].median())

### SEASONALITY
signal_df['Sea_month_5yr_std'] = signal_df['Sea_month_5yr_std'].fillna(signal_df['Sea_month_5yr_std'].median())
### SHORT TERM IVOL
signal_df['idio_vol_20day_adj'] = signal_df['idio_vol_20day_adj'].fillna(signal_df['idio_vol_20day'].median())
### MAX RET
signal_df['maxret_5days_adj'] = signal_df['maxret_5days_adj'].fillna(signal_df['maxret_5days_adj'].median())
# Ear 
signal_df['EAR_std'] = signal_df['EAR_std'].fillna(signal_df['EAR_std'].median())
# Liquidity Shock
signal_df['liq_shock'] = signal_df['liq_shock'].fillna(signal_df['liq_shock'].median())



######### RANK ON INDIVIDUAL PREDICTORS
signal_df['Res_Mom_1M_adj_rank'] = signal_df['Res_Mom_1M_adj'].rank(ascending=True, pct = True)
#high is good
signal_df['Sector Momentum Rank'] =  signal_df['Sector Weighted Mom'].rank(ascending=False, pct = True)
#high is good
signal_df['Seasonality Rank'] =  signal_df['Sea_month_5yr'].rank(ascending=False, pct = True)
#high is good
signal_df['Seasonality Rank Std'] =  signal_df['Sea_month_5yr_std'].rank(ascending=False, pct = True)
#low is good
signal_df['IVOL_adj Rank'] =  signal_df['idio_vol_20day_adj'].rank(ascending=True, pct = True)
#low is good
signal_df['MAXRET_adj Rank'] =  signal_df['maxret_5days_adj'].rank(ascending=True, pct = True)
#high is good
signal_df['EAR_std Rank'] =  signal_df['EAR_std'].rank(ascending=False, pct = True)
#high is good
signal_df['liq_shock Rank'] =  signal_df['liq_shock'].rank(ascending=False, pct = True)





################# YOU WANT THE LOWEST SCORE POSSIBLE
## USE PCT. 
## Implement an interaction score for liquidity and SREV 

signal_df['High_Freq_Combo'] = ( signal_df['Sector Momentum Rank'] +
                               signal_df['Res_Mom_1M_adj_rank'] +
                               signal_df['Seasonality Rank Std'] +  signal_df['EAR_std Rank'] +
                                signal_df['liq_shock Rank']+
                                0.5*signal_df['IVOL_adj Rank'] + 0.5*signal_df['MAXRET_adj Rank']
                                ).rank(ascending=True)



signal_df['Signal'] = "Neutral"
idx_BUY = signal_df['High_Freq_Combo']<=signal_df['High_Freq_Combo'].quantile(CUTOFF)
signal_df.loc[idx_BUY,'Signal']= 'Buy'


idx_SELL = signal_df['High_Freq_Combo']>=signal_df['High_Freq_Combo'].quantile(1-CUTOFF)
signal_df.loc[idx_SELL,'Signal'] = 'Sell'
rank_data = signal_df



In [31]:
folder = "../clean_equity_data/"

port_file = ("../portfolios/eriks_port.xlsx")
SPREAD = 3

current_port_tmp = pd.read_excel(port_file)
current_port_tmp = current_port_tmp.loc[current_port_tmp['Current %']>0,]
current_port_tmp = current_port_tmp.loc[~current_port_tmp['Company'].isin(["Cash", "Total"])]
current_port = current_port_tmp['Company']



# file_list = ["GESTALT_2022-06-29.csv","GESTALT_2022-07-28.csv", "GESTALT_2022-08-27.csv" ]
# N_stocks = [20,20,15]


file_list =['GESTALT_2022-08-27.csv', 'GESTALT_2022-08-27.csv', 'GESTALT_2022-08-27.csv']
N_stocks = [10, 10, 10]



In [32]:
current_port

0                Arctic Paper
1                    Dedicare
2                   Rottneros
3     International Petroleum
4                      SSAB B
5                     EnQuest
6               B3 Consulting
7                    New Wave
8                     Betsson
9                      Prevas
11              Nilörngruppen
13                   BE Group
14                  TietoEVRY
17                Clas Ohlson
18                  Softronic
19                 Africa Oil
20                      Bilia
Name: Company, dtype: object

In [33]:
port_tmp = pd.DataFrame()

for file in file_list:
    N = N_stocks[file_list.index(file)]
    data_tmp = pd.read_csv(folder + file)
    buy = data_tmp[0:N][['Company','Yahoo' ,'Gestalt Rank', 'Sector']]
    
    #ONLY KEEP THE HOLD SPREAD FOR THE LAST MONTH? 
    hold = data_tmp[N: round(SPREAD*N)][['Company','Yahoo', 'Gestalt Rank']] # update to latest month spread??
    keep = hold[hold['Company'].isin(current_port)]
    
    opt_port = pd.concat([buy,keep])
    opt_port.loc[: ,'Weight'] = ERC_gestalt(opt_port, lookback=126)
    port_tmp = pd.concat([port_tmp,opt_port])
    
comp_data = port_tmp.drop_duplicates(subset=['Company'])   
port_tmp = port_tmp.groupby(['Company']).sum()[['Weight']]

Iteration limit exceeded    (Exit mode 9)
            Current function value: 1.064682671686621e-09
            Iterations: 100001
            Function evaluations: 2813918
            Gradient evaluations: 100000
Iteration limit exceeded    (Exit mode 9)
            Current function value: 1.0646826729955636e-09
            Iterations: 100001
            Function evaluations: 2819358
            Gradient evaluations: 100000
Iteration limit exceeded    (Exit mode 9)
            Current function value: 1.0646825393060503e-09
            Iterations: 100001
            Function evaluations: 2803583
            Gradient evaluations: 100000


In [16]:
### SELL THESE HOLDINGS
SELL_LIST =pd.DataFrame(list(set(current_port) - set(port_tmp.index)),columns = ['Company'] )
SELL_LIST = SELL_LIST.groupby(['Company']).sum()
#Buy these holding
BUY_LIST = pd.DataFrame(list(set(port_tmp.index) - set(current_port)),columns = ['Company'])
BUY_LIST = BUY_LIST.groupby(['Company']).sum()


port_tmp.loc[: ,'Weight'] = (port_tmp['Weight']*100).round(decimals=2)
FINAL_PORT = pd.DataFrame(port_tmp['Weight'])
FINAL_PORT = FINAL_PORT.sort_values(by = 'Weight', ascending=False)
FINAL_PORT['Weight'] = (FINAL_PORT['Weight']/FINAL_PORT['Weight'].sum()).apply(lambda x: round_to_multiple(x, 0.005))


In [17]:
FULL_PORT = pd.concat([SELL_LIST,FINAL_PORT]).merge(rank_data[['Company','Signal', "Ticker", "Yahoo"]], on = 'Company')
FULL_PORT = FULL_PORT.sort_values(by = 'Company', ascending=True)

NEW_PORT = FULL_PORT.merge(current_port_tmp[['Company','Antal' ]], on = 'Company', how = 'outer')
NEW_PORT.loc[NEW_PORT['Antal'].isna(),'Antal'] = 0
NEW_PORT.loc[NEW_PORT['Weight'].isna(),'Weight'] = 0



In [18]:
NEW_PORT

,Company,Weight,Signal,Ticker,Yahoo,Antal
0,Africa Oil,0.050,Buy,AOI,AOI.ST,617.0
1,Arctic Paper,0.050,Buy,ARP,ARP.ST,269.0
2,B3 Consulting,0.050,Buy,B3,B3.ST,72.0
3,BE Group,0.040,Buy,BEGR,BEGR.ST,84.0
4,Betsson,0.075,Buy,BETS B,BETS-B.ST,247.0
5,Bilia,0.070,Sell,BILI A,BILI-A.ST,80.0
6,Clas Ohlson,0.065,Buy,CLAS B,CLAS-B.ST,174.0
7,Dedicare,0.075,Neutral,DEDI,DEDI.ST,164.0
8,EnQuest,0.050,Buy,ENQ,ENQ.ST,3479.0
9,International Petroleum,0.055,Buy,IPCO,IPCO.ST,114.0


In [34]:
SELL_LIST

""
Company
Björn Borg
Kabe
Poolia


In [35]:
BUY_LIST

""
Company
International Petroleum


In [36]:
prices_df = pd.DataFrame()
for tick in FULL_PORT['Yahoo']:
    
    price = yf.download(tick,start='2000-01-01', progress = False, threads = False)
    price = price['Adj Close']
    prices_df[tick] = price
    
log_ret = np.log(prices_df) - np.log(prices_df.shift(1))
log_ret = log_ret.dropna() 

hist_port = log_ret.mul(FULL_PORT['Weight'].values, axis=1).sum(axis = 1).tail(252)

prices_df = pd.DataFrame()
for tick in ["^OMX"]:
    
    price = yf.download(tick,start='2000-01-01', progress = False, threads = False)
    price = price['Adj Close']
    prices_df[tick] = price
log_ret_tmp = np.log(prices_df) - np.log(prices_df.shift(1))
log_ret_tmp = log_ret_tmp.dropna()

from sklearn.linear_model import LinearRegression

y = hist_port
X = np.array(log_ret_tmp.tail(252))#.reshape(-1, 1)
reg = LinearRegression().fit(X, y)

In [37]:
date_signal = pd.DataFrame([dparser.parse(latest_file,fuzzy=True).strftime("%d/%m/%Y")])
date_signal.columns = ["Date of Signal"]

beta = pd.DataFrame([reg.coef_])
beta.columns = ["Beta to OMX"]
vol = pd.DataFrame([hist_port.tail(60).std() * np.sqrt(252)])
vol.columns = ["Historical Vol"]

In [123]:
clean_write = pd.DataFrame([])

Zeros =  [0] * N_stocks[0]*2
Blanks = [" "] * N_stocks[0]*2

clean_write.loc[:,'Antal'] = Zeros
clean_write.loc[:,'Weight'] = Zeros
clean_write.loc[:,'Company'] = Blanks
clean_write.loc[:,'Ticker'] = Blanks
clean_write.loc[:,'Signal'] = Blanks

In [124]:
#### Write to excel file
#current_port_tmp

from openpyxl import load_workbook
book = load_workbook(port_file)
writer = pd.ExcelWriter(port_file, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

clean_write
## CLEAN OLD FILE
clean_write.to_excel(writer, "Gestalt",columns=['Company'], index = False, startcol = 1)
clean_write.to_excel(writer, "Gestalt",columns=['Ticker'], index = False, startcol = 2)
clean_write.to_excel(writer, "Gestalt",columns=['Antal'], index = False, startcol = 3)
clean_write.to_excel(writer, "Gestalt",columns=['Weight'], index = False, startcol = 4)
clean_write.to_excel(writer, "Gestalt",columns=['Signal'], index = False, startcol = 10)


## WRITE NEW PORTFOLIO TO FILE
NEW_PORT.to_excel(writer, "Gestalt",columns=['Company'], index = False, startcol = 1)
NEW_PORT.to_excel(writer, "Gestalt",columns=['Ticker'], index = False, startcol = 2)
NEW_PORT.to_excel(writer, "Gestalt",columns=['Antal'], index = False, startcol = 3)
NEW_PORT.to_excel(writer, "Gestalt",columns=['Weight'], index = False, startcol = 4)
NEW_PORT.to_excel(writer, "Gestalt",columns=['Signal'], index = False, startcol = 10)


date_signal.to_excel(writer, "Gestalt", index = False, startcol = 10, startrow=N_stocks[0]*2 +1)
beta.to_excel(writer, "Gestalt", index = False, startcol = 11, startrow=N_stocks[0]*2 +1)
vol.to_excel(writer, "Gestalt", index = False, startcol = 12, startrow=N_stocks[0]*2 +1)
writer.save()

In [125]:
############### EXPERIMENT ###################